In [ ]:
feeding_hours = {
    'seglberget_pen_id_66_2020-05-13_2020-06-13': None,
    'bolaks_pen_id_88_2020-02-28_2020-03-10': None,
    'langoy_pen_id_108_2020-05-07_2020-05-17': None,
    'tittelsnes_pen_id_37_2020-06-10_2020-06-24': None,
    'aplavika_pen_id_95_2020-07-10_2020-07-26': None,
#     'kjeppevikholmen_pen_id_5_2019-06-18_2019-07-02': 'sexton',
    'silda_pen_id_86_2020-07-02_2020-07-19': None,
    'vikane_pen_id_60_2020-08-10_2020-08-30': None,
    'eldviktaren_pen_id_164_2020-09-21_2020-10-08': [7, 15],
#     'habranden_pen_id_100_2020-08-10_2020-08-31': 'imenco',
    'varholmen_pen_id_131_2020-08-15_2020-08-30': None,
    'dale_pen_id_143_2020-10-07_2020-10-21': None,
    'djubawik_pen_id_153_2020-11-10_2020-11-26': [7, 15],
    'leivsethamran_pen_id_165_2020-10-18_2020-11-13': [7, 15],
    'movikodden_pen_id_114_2020-11-03_2020-11-25': [7, 15],
    'movikodden_pen_id_167_2020-10-13_2020-10-30': [6, 15],
    'slapoya_pen_id_116_2020-10-18_2020-11-08': [7, 15],
    'varholmen_pen_id_151_2020-10-02_2020-10-17': [7, 17],
    'varholmen_pen_id_186_2020-10-18_2020-11-02': [6, 14]
}

In [ ]:
import json
import os
import pandas as pd
from research.utils.data_access_utils import S3AccessUtils
from report_generation.report_generator import generate_ts_data, SamplingFilter
from research.utils.datetime_utils import add_days
from report_generation.report_generator import gen_pm_base
from population_metrics.smart_metrics import generate_smart_avg_weight, generate_smart_individual_values, ValidationError
from filter_optimization.filter_optimization_task import _add_date_hour_columns
from research.weight_estimation.keypoint_utils.optics import pixel2world
import numpy as np

pd.set_option('display.max_rows', 500)

In [ ]:
s3 = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))

In [ ]:
cohort_names = [
    'seglberget_pen_id_66_2020-05-13_2020-06-13',
    'bolaks_pen_id_88_2020-02-28_2020-03-10',
    'langoy_pen_id_108_2020-05-07_2020-05-17',
    'tittelsnes_pen_id_37_2020-06-10_2020-06-24',
    'aplavika_pen_id_95_2020-07-10_2020-07-26',
#     'kjeppevikholmen_pen_id_5_2019-06-18_2019-07-02',
    'silda_pen_id_86_2020-07-02_2020-07-19',
    'vikane_pen_id_60_2020-08-10_2020-08-30',
    'eldviktaren_pen_id_164_2020-09-21_2020-10-08',
#     'habranden_pen_id_100_2020-08-10_2020-08-31',
    'varholmen_pen_id_131_2020-08-15_2020-08-30',
    'dale_pen_id_143_2020-10-07_2020-10-21',
    'djubawik_pen_id_153_2020-11-10_2020-11-26',
    'leivsethamran_pen_id_165_2020-10-18_2020-11-13',
    'movikodden_pen_id_114_2020-11-03_2020-11-25',
    'movikodden_pen_id_167_2020-10-13_2020-10-30',
    'slapoya_pen_id_116_2020-10-18_2020-11-08',
    'varholmen_pen_id_151_2020-10-02_2020-10-17',
    'varholmen_pen_id_186_2020-10-18_2020-11-02'
]

In [ ]:
camera_type = {
    'seglberget_pen_id_66_2020-05-13_2020-06-13': 'sexton',
    'bolaks_pen_id_88_2020-02-28_2020-03-10': 'sexton',
    'langoy_pen_id_108_2020-05-07_2020-05-17': 'sexton',
    'tittelsnes_pen_id_37_2020-06-10_2020-06-24': 'sexton',
    'aplavika_pen_id_95_2020-07-10_2020-07-26': 'sexton',
#     'kjeppevikholmen_pen_id_5_2019-06-18_2019-07-02': 'sexton',
    'silda_pen_id_86_2020-07-02_2020-07-19': 'sexton',
    'vikane_pen_id_60_2020-08-10_2020-08-30': 'atlas',
    'eldviktaren_pen_id_164_2020-09-21_2020-10-08': 'atlas',
#     'habranden_pen_id_100_2020-08-10_2020-08-31': 'imenco',
    'varholmen_pen_id_131_2020-08-15_2020-08-30': 'imenco',
    'dale_pen_id_143_2020-10-07_2020-10-21': 'atlas',
    'djubawik_pen_id_153_2020-11-10_2020-11-26': 'atlas',
    'leivsethamran_pen_id_165_2020-10-18_2020-11-13': 'atlas',
    'movikodden_pen_id_114_2020-11-03_2020-11-25': 'imenco',
    'movikodden_pen_id_167_2020-10-13_2020-10-30': 'imenco',
    'slapoya_pen_id_116_2020-10-18_2020-11-08': 'imenco',
    'varholmen_pen_id_131_2020-08-15_2020-08-30': 'imenco',
    'varholmen_pen_id_151_2020-10-02_2020-10-17': 'imenco',
    'varholmen_pen_id_186_2020-10-18_2020-11-02': 'atlas',
    
    
    'dale_pen_id_144_2020-12-20_2021-01-11': 'atlas'
}

In [ ]:
batch_name = 'test'

ROOT_DIR = '/root/data/alok/biomass_estimation/playground'
dfs, gt_metadatas = {}, {}
for cohort_name in cohort_names:
    s3_dir = os.path.join(
        'https://aquabyte-images-adhoc.s3-eu-west-1.amazonaws.com/alok/production_datasets',
        cohort_name
    )

    ground_truth_metadata_url = os.path.join(s3_dir, 'ground_truth_metadata.json')
    ground_truth_key_base = os.path.join(batch_name, cohort_name, 'ground_truth_metadata.json')
#     ground_truth_metadata_url = os.path.join(s3_dir, 'ground_truth_metadata_validated.json')
#     ground_truth_key_base = os.path.join(batch_name, cohort_name, 'ground_truth_metadata_validated.json')
    ground_truth_f = os.path.join(ROOT_DIR, ground_truth_key_base)
    print(ground_truth_metadata_url)
    s3.download_from_url(ground_truth_metadata_url, custom_location=ground_truth_f)
    gt_metadata = json.load(open(ground_truth_f))
    gt_metadatas[cohort_name] = gt_metadata
    
    data_url = os.path.join(s3_dir, 'annotation_dataset.csv')
    data_f, _, _= s3.download_from_url(data_url)
    df = pd.read_csv(data_f)
    df = _add_date_hour_columns(df)
    dfs[cohort_name] = df
    
    

In [ ]:

    
"""This module contains utility helper functions for the WeightEstimator class."""

from collections import namedtuple
from typing import Dict, List, Tuple
import numpy as np
import torch
from research.weight_estimation.keypoint_utils import body_parts


CameraMetadata = namedtuple('CameraMetadata',
                            ['focal_length', 'focal_length_pixel', 'baseline_m',
                             'pixel_count_width', 'pixel_count_height', 'image_sensor_width',
                             'image_sensor_height'])


def get_left_right_keypoint_arrs(annotation: Dict[str, List[Dict]]) -> Tuple:
    """Gets numpy array of left and right keypoints given input keypoint annotation.
    Args:
        annotation: dict with keys 'leftCrop' and 'rightCrop'. Values are lists where each element
        is a dict with keys 'keypointType', 'xCrop' (num pixels from crop left edge),
        'yCrop' (num pixels from crop top edge), 'xFrame' (num pixels from full frame left edge),
        and 'yFrame' (num pixels from full frame top edge).
    Returns:
        X_left: numpy array containing left crop (xFrame, yFrame) for each key-point ordered
        alphabetically.
        X_right: same as above, but for right crop.
    """

    left_keypoints, right_keypoints = {}, {}
    for item in annotation['leftCrop']:
        body_part = item['keypointType']
        left_keypoints[body_part] = (item['xFrame'], item['yFrame'])

    for item in annotation['rightCrop']:
        body_part = item['keypointType']
        right_keypoints[body_part] = (item['xFrame'], item['yFrame'])

    left_keypoint_arr, right_keypoint_arr = [], []
    for body_part in body_parts.core_body_parts:
        left_keypoint_arr.append(left_keypoints[body_part])
        right_keypoint_arr.append(right_keypoints[body_part])

    X_left = np.array(left_keypoint_arr)
    X_right = np.array(right_keypoint_arr)
    return X_left, X_right


def normalize_left_right_keypoint_arrs(X_left: np.ndarray, X_right: np.ndarray) -> Tuple:
    """Normalizes input left and right key-point arrays. The normalization involves (1) 2D
    translation of all keypoints such that they are centered, (2) rotation of the 2D coordiantes
    about the center such that the line passing through UPPER_LIP and fish center is horizontal.
    """

    # translate key-points, perform reflection if necessary
    upper_lip_idx = body_parts.core_body_parts.index(body_parts.UPPER_LIP)
    tail_notch_idx = body_parts.core_body_parts.index(body_parts.TAIL_NOTCH)
    if X_left[upper_lip_idx, 0] > X_left[tail_notch_idx, 0]:
        X_center = 0.5 * (np.max(X_left, axis=0) + np.min(X_left, axis=0))
        X_left_centered = X_left - X_center
        X_right_centered = X_right - X_center
    else:
        X_center = 0.5 * (np.max(X_right, axis=0) + np.min(X_right, axis=0))
        X_left_centered = X_right - X_center
        X_right_centered = X_left - X_center
        X_left_centered[:, 0] = -X_left_centered[:, 0]
        X_right_centered[:, 0] = -X_right_centered[:, 0]

    # rotate key-points
    upper_lip_x, upper_lip_y = tuple(X_left_centered[upper_lip_idx])
    theta = np.arctan(upper_lip_y / upper_lip_x)
    R = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta), np.cos(theta)]
    ])

    D = X_left_centered - X_right_centered
    X_left_rot = np.dot(X_left_centered, R)
    X_right_rot = X_left_rot - D
    return X_left_rot, X_right_rot


def convert_to_world_point_arr(X_left: np.ndarray, X_right: np.ndarray,
                               camera_metadata: CameraMetadata) -> np.ndarray:
    """Converts input left and right normalized keypoint arrays into world coordinate array."""

    y_world = camera_metadata.focal_length_pixel * camera_metadata.baseline_m / \
              (X_left[:, 0] - X_right[:, 0])

    # Note: the lines commented out below are technically the correct formula for conversion
    # x_world = X_left[:, 0] * y_world / camera_metadata.focal_length_pixel
    # z_world = -X_left[:, 1] * y_world / camera_metadata.focal_length_pixel
    x_world = ((X_left[:, 0] * camera_metadata.image_sensor_width / camera_metadata.pixel_count_width) * y_world) / (camera_metadata.focal_length)
    z_world = (-(X_left[:, 1] * camera_metadata.image_sensor_height / camera_metadata.pixel_count_height) * y_world) / (camera_metadata.focal_length)
    X_world = np.vstack([x_world, y_world, z_world]).T
    return X_world


def stabilize_keypoints(X: np.ndarray) -> np.ndarray:
    """Transforms world coordinate array so that neural network inputs are stabilized"""
    X_new = np.zeros(X.shape)
    X_new[:, 0] = 0.5 * X[:, 0] / X[:, 1]
    X_new[:, 1] = 0.5 * X[:, 2] / X[:, 1]
    X_new[:, 2] = 0.05 / X[:, 1]
    return X_new


def convert_to_nn_input(annotation: Dict[str, List[Dict]], camera_metadata: CameraMetadata) \
        -> torch.Tensor:
    """Convrts input keypoint annotation and camera metadata into neural network tensor input."""
    X_left, X_right = get_left_right_keypoint_arrs(annotation)
    X_left_norm, X_right_norm = normalize_left_right_keypoint_arrs(X_left, X_right)
    X_world = convert_to_world_point_arr(X_left_norm, X_right_norm, camera_metadata)
    X = stabilize_keypoints(X_world)
    nn_input = torch.from_numpy(np.array([X])).float()
    return nn_input


"""
This module contains the WeightEstimator class for estimating fish weight (g), length (mm), and
k-factor given input keypoint coordinates and camera metadata.
"""

from typing import Dict, Tuple
import torch
from torch import nn


class Network(nn.Module):
    """Network class defines neural-network architecture for both weight and k-factor estimation
    (currently both neural networks share identical architecture)."""

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(24, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        """Run inference on input keypoint tensor."""
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.output(x)
        return x


class WeightEstimator:
    """WeightEstimator class is used to predict fish weight, k-factor, and length
    given input keypoint annotations and camera metadata."""

    def __init__(self, weight_model_f: str, kf_model_f: str) -> None:
        """Initializes class with input weight and k-factor neural-networks."""
        self.weight_model = Network()
        self.weight_model.load_state_dict(torch.load(weight_model_f))
        self.weight_model.eval()

        self.kf_model = Network()
        self.kf_model.load_state_dict(torch.load(kf_model_f))
        self.kf_model.eval()

    @staticmethod
    def _get_model_input(annotation: Dict, camera_metadata: CameraMetadata) -> torch.Tensor:
        """Generates neural-network input tensor given annotation and camera_metadata."""
        X = convert_to_nn_input(annotation, camera_metadata)
        return X

    def predict_weight(self, annotation: Dict, camera_metadata: CameraMetadata) -> float:
        """Generates weight prediction given input annotation and camera metadata."""
        X = self._get_model_input(annotation, camera_metadata)
        weight = 1e4 * self.weight_model(X).item()
        return weight

    def predict_kf(self, annotation: Dict, camera_metadata: CameraMetadata) -> float:
        """Generates k-factor prediction gievn input annotation and camera metadata."""
        X = self._get_model_input(annotation, camera_metadata)
        kf = self.kf_model(X).item()
        return kf

    def predict(self, annotation: Dict, camera_metadata: CameraMetadata) -> Tuple:
        """Generates weight, k-factor, and length predictions given input annotation and camera
        metadata."""
        weight = self.predict_weight(annotation, camera_metadata)
        kf = self.predict_kf(annotation, camera_metadata)
        if weight * kf > 0:
            length = (1e5 * weight / kf) ** (1.0 / 3)
        else:
            length = 0
        return weight, length, kf

In [ ]:
models = [
    ('weight_v1', 'curr-nonsynthetic', 'https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/playground/nn_epoch_798_v2.pb', True),
    ('weight_v2', 'curr-synthetic', 'https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2020-11-27T00-00-00/weight_model_synthetic_data.pb', True)
]

In [ ]:
for key, tag, model_url, is_url in models:
    # weight_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2020-11-27T00-00-00/weight_model_synthetic_data.pb')
    if is_url:
        weight_model_f, _, _ = s3.download_from_url(model_url)
    else:
        weight_model_f = model_url
    kf_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/playground/kf_predictor_v2.pb')

    weight_estimator = WeightEstimator(weight_model_f, kf_model_f)


    for k, rdf in dfs.items():
        print(k)
        weights = []
        count = 0
        for idx, row in rdf.iterrows():
            if count % 100 == 0:
                print('Percentage completion: {}%'.format(round(100 * count / rdf.shape[0], 2)))
                print(count)
            count += 1
            annotation = json.loads(row.annotation.replace("'", '"'))
            if not annotation:
                weights.append(None)
                continue
            camera_metadata = json.loads(row.camera_metadata.replace("'", '"'))
            if not camera_metadata:
                camera_metadata = json.loads(rdf.camera_metadata.iloc[0].replace("'", '"'))

            camera_metadata_obj = CameraMetadata(
                focal_length=camera_metadata['focalLength'],
                focal_length_pixel=camera_metadata['focalLengthPixel'],
                baseline_m=camera_metadata['baseline'],
                pixel_count_width=camera_metadata['pixelCountWidth'],
                pixel_count_height=camera_metadata['pixelCountHeight'],
                image_sensor_width=camera_metadata['imageSensorWidth'],
                image_sensor_height=camera_metadata['imageSensorHeight']
            )

            weight, length, kf = weight_estimator.predict(annotation, camera_metadata_obj)
            weights.append(weight)
        rdf[key] = weights


In [ ]:
def generate_raw_individual_values(pm_base, gt_metadata, start_hour, end_hour, apply_growth_rate, max_day_diff, days_post_feeding, final_days_post_feeding):
    last_feeding_date = gt_metadata['last_feeding_date']
    date = add_days(last_feeding_date, days_post_feeding)
    weights, _ = generate_smart_individual_values(pm_base, date, max_day_diff, True, apply_growth_rate, 0.9)
    return weights


def generate_average_weight_accuracy(weights, gt_metadata, loss_factor):
    avg_weight_prediction = np.mean(weights)
    gutted_weight_prediction = avg_weight_prediction * (1.0 - loss_factor)
    gt_weight = gt_metadata['gutted_average_weight']
    avg_weight_err = (gutted_weight_prediction - gt_weight) / gt_weight
    return avg_weight_err, gutted_weight_prediction

def generate_distribution_accuracy(weights, gt_metadata, loss_factor):
    gutted_weights = weights * (1.0 - loss_factor)
    gutted_weight_distribution = gt_metadata['gutted_weight_distribution']
    
    if gutted_weight_distribution is None:
        return []
    
    count_distribution_errors = []
    
    for bucket in gutted_weight_distribution:
        lower_bound, upper_bound = bucket.split('-')
        pct = gutted_weight_distribution[bucket]
        mask = (gutted_weights >= float(lower_bound) * 1000) & (gutted_weights < float(upper_bound) * 1000)

        pct = np.sum(mask) / len(mask)
        gt_pct = gutted_weight_distribution[bucket] / 100
        
        count_distribution_errors.append(pct - gt_pct)
        
    return count_distribution_errors



In [ ]:
all_dfs1 = []
all_dfs2 = []
all_dfs3 = []
all_dfs4 = []
all_dfs5 = []
all_dfs6 = []

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt

for key, tag, _, _ in models:
    start_hours = [7]
    end_hours = [15]
    apply_growth_rate = True
    max_day_diff = 3
    days_post_feeding = 1
    final_days_post_feeding = 1
    loss_factors = [0.16, 'expected_loss_factor'] # need to determine the right values here
    akpd_cutoffs = [0.01, 0.95]

    hour_filter_methods = ['manual', 'hour_hist', 'u-shape'] #  'u-shape',

    cohort_name_col = []
    akpd_cutoff_col = []
    hour_filter_method_col = []
    start_hour_col = []
    end_hour_col = []
    loss_factor_col = []
    starvation_days_col = []
    avg_weight_col = []
    avg_weight_error_col = []
    gt_avg_weight_col = []
    count_distribution_error_col = []
    camera_col = []
    gt_feeding_hour_start_col = []
    gt_feeding_hour_end_col = []

    for loss_factor in loss_factors:
        avg_weight_error_col.append([])
        avg_weight_error_col.append([])
        count_distribution_error_col.append([])

    for cohort_name in sorted(list(dfs.keys())):
        print(cohort_name)
        
        gt_metadata = gt_metadatas[cohort_name]

        last_feeding_date = gt_metadata['last_feeding_date']
        slaughter_date = gt_metadata['slaughter_date']

        if slaughter_date is not None and last_feeding_date is not None:
            date_diff = datetime.strptime(slaughter_date, '%Y-%m-%d') - datetime.strptime(last_feeding_date, '%Y-%m-%d')
            starvation_days = date_diff.days
        else:
            starvation_days = None

        df = dfs[cohort_name]
        df['estimated_weight_g'] = df[key]
        final_date_post_feeding = add_days(gt_metadata['last_feeding_date'], final_days_post_feeding)
        tdf = df[df.date <= final_date_post_feeding]

        start_end_hours = []

        for method in hour_filter_methods:
            if method == 'manual':
                for start_hour in start_hours:
                    for end_hour in end_hours:
                        start_end_hours.append((method, start_hour, end_hour))
            elif method == 'u-shape':
                df2 = df[(df.hour >= 3) & (df.hour <= 20)]

                #count, bins, _ = plt.hist(df2.hour, density = True, bins = (np.max(df2.hour) - np.min(df2.hour)))

                start_hour = np.min(df2.hour)
                end_hour = np.max(df2.hour)

                bins = np.arange(start_hour, end_hour + 1)

                weights = []

                for hour in np.arange(start_hour, end_hour + 1):
                    avg_weight = np.mean(df2[df2.hour == hour].estimated_weight_g)
                    weights.append(avg_weight)

                start_index = np.where(bins == 10)[0][0]

                lower_index = start_index
                upper_index = start_index

                is_iterating_lower = True
                is_iterating_upper = True
                eps_lower = 3
                eps_upper = 6

                while is_iterating_lower or is_iterating_upper:
                #     print(lower_index, weights[lower_index], upper_index, weights[upper_index], np.std(weights[lower_index:upper_index]))
                #     print(np.take(weights, np.arange(lower_index, upper_index + 1, 1)))
                #     if lower_index > 0 and upper_index < len(weights) - 1 and np.all(np.abs(np.diff(np.take(weights, np.arange(lower_index - 1, upper_index + 1, 2)))) < min(max(eps * np.std(weights[lower_index:upper_index]), 100), 200)):
                #         lower_index = lower_index - 1
                #         upper_index = upper_index + 1
                    hour_lower = bins[lower_index]
                    hour_upper = bins[upper_index]
                    hour_filter = (df2.hour >= hour_lower) & (df2.hour <= hour_upper)

                    sd = np.std(df2[hour_filter].estimated_weight_g) / np.sqrt(np.sum(hour_filter))

                    lower_limit = max(sd * eps_lower, 150)
                    upper_limit = max(sd * eps_upper, 150)

                    if is_iterating_lower and lower_index > 0:
                        vals1 = np.abs(np.diff(np.take(weights, np.arange(lower_index - 1, upper_index + 1, 1))))
                        vals2 = np.abs(np.diff(np.take(weights, np.arange(lower_index - 1, upper_index + 1, 2))))

                        limit = min(max(eps * np.std(weights[lower_index:(upper_index + 1)]), lower_limit), upper_limit)

                        if np.all(vals1 < limit) and np.all(vals2 < limit):
                            lower_index = lower_index - 1
                            is_iterating_upper = True
                        else:
                            is_iterating_lower = False
                    elif is_iterating_upper and upper_index < len(weights) - 1:
                        vals1 = np.abs(np.diff(np.take(weights, np.arange(lower_index, upper_index + 2, 1))))
                        if (upper_index + 1 - lower_index) % 2 == 0: 
                            vals2 = np.abs(np.diff(np.take(weights, np.arange(lower_index, upper_index + 2, 2))))
                        else:
                            vals2 = np.abs(np.diff(np.take(weights, np.arange(lower_index + 1, upper_index + 2, 2))))

                        limit = min(max(eps * np.std(weights[lower_index:(upper_index + 1)]), lower_limit), upper_limit)

                        if np.all(vals1 < limit) and np.all(vals2 < limit):
                            upper_index = upper_index + 1
                            is_iterating_lower = True
                        else:
                #             print(vals1)
                #             print(vals2)
                            is_iterating_upper = False
                    else:
                        is_iterating_lower = False
                        is_iterating_upper = False
                
                start_hour, end_hour = bins[lower_index], bins[upper_index]
                
                start_end_hours.append((method, start_hour, end_hour))
            elif method == 'hour_hist':
                df2 = df[(df.hour >= 3) & (df.hour <= 20)]

                count, bins, _ = plt.hist(df2.hour, density = True, bins = (np.max(df2.hour) - np.min(df2.hour)))

                idx_values = np.where(count > 1.0 / 18)[0]

                start_index = np.where(bins == 10)[0][0]
                start_array = np.where(idx_values == start_index)[0][0]

                lower_index = start_array
                upper_index = start_array

                while lower_index > 0 and (idx_values[lower_index] - idx_values[lower_index - 1] == 1):
                    lower_index = lower_index - 1
                while upper_index < len(idx_values) - 1 and (idx_values[upper_index + 1] - idx_values[upper_index] == 1):
                    upper_index = upper_index + 1

                start_hour, end_hour = bins[idx_values[lower_index]], bins[idx_values[upper_index]]

                start_end_hours.append((method, start_hour, end_hour))

        for akpd_cutoff in akpd_cutoffs:
            for method, start_hour, end_hour in start_end_hours:
                sampling_filter = SamplingFilter(
                    start_hour=start_hour,
                    end_hour=end_hour,
                    kf_cutoff=0.0,
                    akpd_score_cutoff=akpd_cutoff
                )

                pm_base = gen_pm_base(tdf, sampling_filter)

                try:
                    weights = generate_raw_individual_values(pm_base, gt_metadata, start_hour, end_hour, apply_growth_rate, max_day_diff, days_post_feeding, final_days_post_feeding)
                except ValidationError as err:
                    continue

                akpd_cutoff_col.append(akpd_cutoff)
                cohort_name_col.append(cohort_name)
                hour_filter_method_col.append(method)
                start_hour_col.append(start_hour)
                end_hour_col.append(end_hour)
                loss_factor_col.append(gt_metadata['expected_loss_factor'])
                starvation_days_col.append(starvation_days)
                avg_weight_col.append(np.mean(weights))
                gt_avg_weight_col.append(gt_metadata['gutted_average_weight'])
                camera_col.append(camera_type[cohort_name])
                
                if feeding_hours[cohort_name] is not None:
                    gt_feeding_hour_start_col.append(feeding_hours[cohort_name][0])
                    gt_feeding_hour_end_col.append(feeding_hours[cohort_name][1])
                else:
                    gt_feeding_hour_start_col.append(0)
                    gt_feeding_hour_end_col.append(0)

                for index, loss_factor in enumerate(loss_factors):
                    if loss_factor == 'expected_loss_factor':
                        loss_factor = gt_metadata['expected_loss_factor'] or 0.165

                        if loss_factor > 10:
                            loss_factor = loss_factor / 100.0

                    avg_weight_err, gutted_weight_prediction = generate_average_weight_accuracy(weights, gt_metadata, loss_factor)
                    avg_weight_error_col[index].append(avg_weight_err)

                    count_distribution_errors = generate_distribution_accuracy(weights, gt_metadata, loss_factor)
                    count_distribution_error_col[index].append(count_distribution_errors)
                    
    columns = {
        'cohort_name': cohort_name_col,
        'hour_filter_method_col': hour_filter_method_col,
        'akpd_cutoff_col': akpd_cutoff_col,
        'start_hour_col': start_hour_col,
        'end_hour_col': end_hour_col,
        'loss_factor_col': loss_factor_col,
        'starvation_days_col': starvation_days_col,
        'avg_weight_col': avg_weight_col,
        'gt_avg_weight_col': gt_avg_weight_col,
        'camera_col': camera_col,
        'gt_feeding_hour_start_col': gt_feeding_hour_start_col,
        'gt_feeding_hour_end_col': gt_feeding_hour_end_col
    }

    for index, loss_factor in enumerate(loss_factors):
        if loss_factor == 'expected_loss_factor':
            col_name = 'avg_weight_error_exp'
            col_abs_name = 'avg_weight_error_abs_exp'
            col_abs_dist_name = 'avg_count_dist_error_abs_exp'
        else:
            col_name = 'avg_weight_error_%0.2f' % (loss_factor,)
            col_abs_name = 'avg_weight_error_abs_%0.2f' % (loss_factor,)
            col_abs_dist_name = 'avg_count_dist_error_abs_%0.2f' % (loss_factor,)

        columns[col_name] = avg_weight_error_col[index]
        columns[col_abs_name] = np.abs(avg_weight_error_col[index])
        columns[col_abs_dist_name] = [np.mean(np.abs(l)) for l in count_distribution_error_col[index]]

    tdf = pd.DataFrame(columns)
    
    reporting_cols = ['cohort_name', 'avg_weight_col', 'gt_avg_weight_col', 'avg_weight_error_0.16', 'avg_weight_error_exp', 'start_hour_col', 'end_hour_col', 'gt_feeding_hour_start_col', 'gt_feeding_hour_end_col']
    
    df1 = tdf[(tdf.akpd_cutoff_col == 0.01) & (tdf.hour_filter_method_col == 'manual')][reporting_cols]
    df2 = tdf[(tdf.akpd_cutoff_col == 0.01) & (tdf.hour_filter_method_col == 'hour_hist')][reporting_cols]
    df3 = tdf[(tdf.akpd_cutoff_col == 0.01) & (tdf.hour_filter_method_col == 'u-shape')][reporting_cols]
    df4 = tdf[(tdf.akpd_cutoff_col == 0.95) & (tdf.hour_filter_method_col == 'manual')][reporting_cols]
    df5 = tdf[(tdf.akpd_cutoff_col == 0.95) & (tdf.hour_filter_method_col == 'hour_hist')][reporting_cols]
    df6 = tdf[(tdf.akpd_cutoff_col == 0.95) & (tdf.hour_filter_method_col == 'u-shape')][reporting_cols]
    
    all_dfs1.append(df1)
    all_dfs2.append(df2)
    all_dfs3.append(df3)
    all_dfs4.append(df4)
    all_dfs5.append(df5)
    all_dfs6.append(df6)

In [ ]:
all_dfs = all_dfs1

metric = []

for index, model in enumerate(models):
    _, tag, _, _ = model
    print('%-*s: %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f' % (25, tag, 100 * np.percentile(np.abs(all_dfs[index]['avg_weight_error_exp']), 90), 100 * np.percentile(np.abs(all_dfs[index]['avg_weight_error_exp']), 50), np.sqrt(np.mean((50 * np.abs(all_dfs[index]['avg_weight_error_exp'])) ** 2)), 100 * np.mean((all_dfs[index]['avg_weight_error_0.16'])), 100 * np.mean(np.abs(all_dfs[index]['avg_weight_error_0.16'])), 100 * np.mean(np.abs(all_dfs[index]['avg_weight_error_exp'])), 100 * np.std((all_dfs[index]['avg_weight_error_0.16']))))
    metric.append(np.sqrt(np.mean((50 * np.abs(all_dfs[index]['avg_weight_error_exp'])) ** 2)))
    
print(np.mean(metric))

In [ ]:
all_dfs = all_dfs2

metric = []

for index, model in enumerate(models):
    _, tag, _, _ = model
    print('%-*s: %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f' % (25, tag, 100 * np.percentile(np.abs(all_dfs[index]['avg_weight_error_exp']), 90), 100 * np.percentile(np.abs(all_dfs[index]['avg_weight_error_exp']), 50), np.sqrt(np.mean((50 * np.abs(all_dfs[index]['avg_weight_error_exp'])) ** 2)), 100 * np.mean((all_dfs[index]['avg_weight_error_0.16'])), 100 * np.mean(np.abs(all_dfs[index]['avg_weight_error_0.16'])), 100 * np.mean(np.abs(all_dfs[index]['avg_weight_error_exp'])), 100 * np.std((all_dfs[index]['avg_weight_error_0.16']))))
    metric.append(np.sqrt(np.mean((50 * np.abs(all_dfs[index]['avg_weight_error_exp'])) ** 2)))
    
print(np.mean(metric))

In [ ]:
all_dfs = all_dfs3

metric = []

for index, model in enumerate(models):
    _, tag, _, _ = model
    print('%-*s: %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f' % (25, tag, 100 * np.percentile(np.abs(all_dfs[index]['avg_weight_error_exp']), 90), 100 * np.percentile(np.abs(all_dfs[index]['avg_weight_error_exp']), 50), np.sqrt(np.mean((50 * np.abs(all_dfs[index]['avg_weight_error_exp'])) ** 2)), 100 * np.mean((all_dfs[index]['avg_weight_error_0.16'])), 100 * np.mean(np.abs(all_dfs[index]['avg_weight_error_0.16'])), 100 * np.mean(np.abs(all_dfs[index]['avg_weight_error_exp'])), 100 * np.std((all_dfs[index]['avg_weight_error_0.16']))))
    metric.append(np.sqrt(np.mean((50 * np.abs(all_dfs[index]['avg_weight_error_exp'])) ** 2)))
    
print(np.mean(metric))

In [ ]:
all_dfs3[-1]

In [ ]:
np.mean(np.abs(all_dfs3[-1]['avg_weight_error_0.16']))

In [ ]:
cohort_name = 'varholmen_pen_id_186_2020-10-18_2020-11-02'#cohort_names[17]

df = dfs[cohort_name]
df2 = df[(df.hour >= 3) & (df.hour <= 20)]

#count, bins, _ = plt.hist(df2.hour, density = True, bins = (np.max(df2.hour) - np.min(df2.hour)))

start_hour = np.min(df2.hour)
end_hour = np.max(df2.hour)

bins = np.arange(start_hour, end_hour + 1)

weights = []
sds = []

for hour in np.arange(start_hour, end_hour + 1):
    avg_weight = np.mean(df2[df2.hour == hour].estimated_weight_g)
    
    
    weights.append(avg_weight)
    sds.append(sd)
    
start_index = np.where(bins == 10)[0][0]

lower_index = start_index
upper_index = start_index

is_iterating_lower = True
is_iterating_upper = True

eps_lower = 3
eps_upper = 6

while is_iterating_lower or is_iterating_upper:
#     print(lower_index, weights[lower_index], upper_index, weights[upper_index], np.std(weights[lower_index:upper_index]))
#     print(np.take(weights, np.arange(lower_index, upper_index + 1, 1)))
#     if lower_index > 0 and upper_index < len(weights) - 1 and np.all(np.abs(np.diff(np.take(weights, np.arange(lower_index - 1, upper_index + 1, 2)))) < min(max(eps * np.std(weights[lower_index:upper_index]), 100), 200)):
#         lower_index = lower_index - 1
#         upper_index = upper_index + 1
    hour_lower = bins[lower_index]
    hour_upper = bins[upper_index]
    hour_filter = (df2.hour >= hour_lower) & (df2.hour <= hour_upper)
    
    sd = np.std(df2[hour_filter].estimated_weight_g) / np.sqrt(np.sum(hour_filter))
    
    lower_limit = max(sd * eps_lower, 150)
    upper_limit = max(sd * eps_upper, 150)
    
    if is_iterating_lower and lower_index > 0:
        vals1 = np.abs(np.diff(np.take(weights, np.arange(lower_index - 1, upper_index + 1, 1))))
        vals2 = np.abs(np.diff(np.take(weights, np.arange(lower_index - 1, upper_index + 1, 2))))
        
        limit = min(max(eps * np.std(weights[lower_index:(upper_index + 1)]), lower_limit), upper_limit)
        
        if np.all(vals1 < limit) and np.all(vals2 < limit):
            lower_index = lower_index - 1
            is_iterating_upper = True
        else:
            is_iterating_lower = False
    elif is_iterating_upper and upper_index < len(weights) - 1:
        vals1 = np.abs(np.diff(np.take(weights, np.arange(lower_index, upper_index + 2, 1))))
        if (upper_index + 1 - lower_index) % 2 == 0: 
            vals2 = np.abs(np.diff(np.take(weights, np.arange(lower_index, upper_index + 2, 2))))
        else:
            vals2 = np.abs(np.diff(np.take(weights, np.arange(lower_index + 1, upper_index + 2, 2))))
        
        limit = min(max(eps * np.std(weights[lower_index:(upper_index + 1)]), lower_limit), upper_limit)
        
        if np.all(vals1 < limit) and np.all(vals2 < limit):
            upper_index = upper_index + 1
            is_iterating_lower = True
        else:
#             print(vals1)
#             print(vals2)
            is_iterating_upper = False
    else:
        is_iterating_lower = False
        is_iterating_upper = False
        
start_hour, end_hour = bins[lower_index], bins[upper_index]

plt.plot(bins, weights)

print(start_hour, end_hour)

In [ ]:
sds

In [ ]:
weights[lower_index+4:upper_index + 1]

# 3 * np.std(weights[lower_index + 4:upper_index])

In [ ]:
cohort_name = cohort_names[17]

df = dfs[cohort_name]
df2 = df[(df.hour >= 3) & (df.hour <= 20)]

count, bins, _ = plt.hist(df2.hour, density = True, bins = (np.max(df2.hour) - np.min(df2.hour)))

idx_values = np.where(count > 1.0 / 18)[0]

start_index = np.where(bins == 10)[0][0]
start_array = np.where(idx_values == start_index)[0][0]

lower_index = start_array
upper_index = start_array

while lower_index > 0 and (idx_values[lower_index] - idx_values[lower_index - 1] == 1):
    lower_index = lower_index - 1
while upper_index < len(idx_values) - 1 and (idx_values[upper_index + 1] - idx_values[upper_index] == 1):
    upper_index = upper_index + 1
    
print(bins[idx_values[lower_index]], bins[idx_values[upper_index]])